# **Progetto: Riassunto e Identificazione delle Attività dalla Trascrizione di un Meeting**

Realizzare uno script in Python che: 1. Analizzi una trascrizione di un meeting fornita. 2. Generi un riassunto conciso dei punti principali discussi. 3. Identifichi e strutturi le attività assegnate a ciascun partecipante.

## **Obiettivo del progetto:**

Realizzare uno script in Python che: </br>

1.   Analizzi una trascrizione di un meeting fornita.
2.   Generi un riassunto conciso dei punti principali discussi.
3.   Identifichi e strutturi le attività assegnate a ciascun partecipante.

## **Benefici del progetto:**

- Risparmio tempo analisi manuale.

- Maggiore accuratezza informazioni estratte.

- Facilitare monitoraggio compiti assegnati.

- Miglioramento efficienza comunicazione team.



---

## **Codice:**

### **Dipendeze:**

In [ ]:
!pip install langchain-openai llama-index requests
!pip install langchain-community

In [ ]:
import requests
from langchain_openai import ChatOpenAI
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.llms.openai import OpenAI
import os

# Imposta API OpenAI
os.environ["OPENAI_API_KEY"] = "INSERISCI_LA_TUA_API"

### **Main**

In [ ]:
url = "https://raw.githubusercontent.com/Profession-AI/progetti-llm/refs/heads/main/Riassunto%20e%20identificazione%20delle%20attivit%C3%A0%20dalla%20trascrizione%20di%20un%20meeting/meeting_transcription.txt"

response = requests.get(url)
if response.status_code != 200:
    raise Exception("Caricamento file non riuscito")

with open("meeting_transcription.txt", "w") as file:
    file.write(response.text)

In [ ]:
# Carico i documenti presenti nella directory corrente
documents = SimpleDirectoryReader(".").load_data()

# Configurazione modello LLM (GPT-4-mini)
llm = OpenAI(model="gpt-4.1-mini", temperature=0.2)
Settings.llm = llm
Settings.chunk_size = 1024

# Creazione dell'indice
index = VectorStoreIndex.from_documents(documents)

# Creazione query engine per interrogare l'indice (metodo RAG)
query_engine = index.as_query_engine()

In [ ]:
prompt_summary = """
Genera un breve riassunto puntato evidenziando chiaramente i punti principali discussi nel meeting fornito nella trascrizione.
"""

prompt_tasks = """
Identifica chiaramente le attività assegnate durante il meeting dalla trascrizione, indicando nome e cognome del responsabile e l'attività stessa nel formato:
- Nome Cognome: Attività assegnata
"""

In [ ]:
# Ottieni riassunto dal modello
print("\n Riassunto del Meeting:")
summary_response = query_engine.query(prompt_summary)
print(summary_response)

# Ottieni attività assegnate dal modello
print("\n Attività Identificate:")
tasks_response = query_engine.query(prompt_tasks)
print(tasks_response)


 Riassunto del Meeting:
- Necessità di un’interfaccia utente più intuitiva per l’inserimento delle fatture.  
- Gestione automatizzata delle scadenze dei pagamenti con promemoria e notifiche.  
- Reportistica personalizzabile con filtri per parametri come intervalli di tempo e categorie di spesa.  
- Invio automatico di email ai fornitori in caso di ritardi nei pagamenti.  
- Archiviazione digitale dei documenti, con possibilità di allegare PDF delle fatture.  
- Implementazione di uno spazio di storage cloud e funzione di ricerca avanzata per i documenti.  
- Redazione di un piano tecnico basato sui requisiti emersi per la successiva approvazione del progetto.

 Attività Identificate:
- Andrea Monti: Integrare un sistema di notifiche e invio email nella piattaforma  
- Andrea Monti: Prendere nota della necessità di uno spazio di storage e gestione documentale  
- Andrea Monti: Redigere un piano tecnico basato sui requisiti emersi durante il meeting


---

## **Spiegazione del codice:**


## Spiegazione del codice: Analisi Meeting con RAG, LangChain e LlamaIndex

### Scelte Tecnologiche e Motivazioni

- **LlamaIndex**: Indicizza i documenti di testo con embeddings e ne permette la ricerca semantica rapida.
- **LangChain**: Gestisce la pipeline e l’integrazione tra retrieval e modello LLM, rendendo la soluzione modulare ed estensibile.
- **OpenAI GPT-4.1-mini**: Modello linguistico potente, ideale per riassunti ed estrazione di informazioni contestualizzate.
- **RAG (Retrieval-Augmented Generation)**: Migliora la precisione delle risposte perché si basa sui dati reali forniti in input, riducendo il rischio di informazioni errate (allucinazioni).

---

### Struttura del Codice

#### 1. Download e salvataggio della trascrizione

Scarica il file dal link e lo salva localmente per l’analisi.
```python
    import requests
    url = "..."
    response = requests.get(url)
    with open("meeting_transcription.txt", "w", encoding="utf-8") as file:
        file.write(response.text)
```

---

#### 2. Caricamento e indicizzazione dei documenti

Crea l’indice vettoriale a partire dai file presenti nella cartella, usando LlamaIndex e imposta il modello GPT-4.

```python
    import requests
    from langchain_openai import ChatOpenAI
    from llama_index import VectorStoreIndex, SimpleDirectoryReader
    from llama_index.llms import OpenAI
    import os

    os.environ["OPENAI_API_KEY"] = "..."
    documents = SimpleDirectoryReader(".").load_data()
    llm = OpenAI(model="gpt-4.1-mini", temperature=0.2)
    Settings.llm = llm
    Settings.chunk_size = 1024
    index = VectorStoreIndex.from_documents(documents)
```

Motivazione:
- Caricamento flessibile di più documenti.
- Indicizzazione e embedding automatico dei testi per la ricerca semantica.
- Possibilità di cambiare facilmente modello o parametri (es. temperatura).

---

#### 3. Creazione del motore di query (RAG)

Si crea il query engine che permette di interrogare l’indice vettoriale.

```python
    query_engine = index.as_query_engine()
```

Motivazione:
Permette di sfruttare Retrieval-Augmented Generation (RAG) per risposte precise e contestualizzate ai dati.

---

#### 4. Prompt separati e generazione output

Vengono definiti prompt specifici per ottenere il riassunto del meeting e la lista delle attività assegnate.

```python
    prompt_summary = "Genera un riassunto puntato dei punti principali discussi nel meeting."
    prompt_tasks = "Identifica le attività assegnate ai partecipanti, specificando nome e compito."

    print("\n Riassunto:")
    print(query_engine.query(prompt_summary))

    print("\n Attività:")
    print(query_engine.query(prompt_tasks))
```

Motivazione:
Prompt separati danno risultati più precisi, facilmente riutilizzabili e validabili nei processi aziendali.

---

### Risultati Ottenuti
- Riassunto chiaro dei principali temi trattati durante il meeting.
- Elenco strutturato delle attività, con indicazione dei responsabili per ogni compito.
- Risposte pertinenti e aderenti al contenuto della trascrizione, con minima possibilità di errori o “fantasie” del modello.

---

### Vantaggi della Soluzione

- Riduce errori e velocizza l’analisi delle trascrizioni.
- Risultati sempre aggiornati, ancorati ai dati reali.
- Facile da integrare in pipeline di automazione o dashboard.
- Possibilità di estensione con prompt o dati aggiuntivi.

---

### Limitazioni del Sistema

- Dipendenza dalla qualità della trascrizione: Se la trascrizione contiene errori o mancano nomi/compiti, anche il sistema produrrà risultati incompleti.
- Necessità di una connessione Internet e di una chiave API per accedere ai modelli OpenAI.
- Non distingue compiti impliciti: Solo le attività esplicitamente menzionate vengono rilevate.
- Costo API: L’utilizzo di modelli GPT può avere un costo se usato su grandi volumi.